In [ ]:
import sys

In [ ]:
# Helper functions

def bxor(b1, b2): # xor two bytes (trims the longer input)
    result = b""
    for b1, b2 in zip(b1, b2):
        result += bytes([b1 ^ b2])
    return result

def is_ascii(hex_char):
    valid_hex_digits = ["0", "1", "2", "3", "4", "5", "6", "7"]
    return (hex_char[0] in valid_hex_digits)

def is_az_char(hex_char):
    az_chars = ["41", "42", "43", "44", "45", "46", "47", "48", "49", "4a", "4b", "4c", "4d", "4e", "4f", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59", "5a",
                "61", "62", "63", "64", "65", "66", "67", "68", "69", "6a", "6b", "6c", "6d", "6e", "6f", "70", "71", "72", "73", "74", "75", "76", "77", "78", "79", "7a"]
    return (hex_char in az_chars)

def replace_non_ascii_chars_with_space(hex_string):
    new_string = ""
    for i in range(int(len(hex_string)/2)):
        if (not is_ascii(hex_string[2*i:2*(i + 1)])):
            new_string += "20"
        else:
            new_string += hex_string[2*i:2*(i + 1)]     
    return new_string

def identity(hex_string):
    return hex_string

def print_text(hex_string, preprocessing_function):
    hex_string = preprocessing_function(hex_string)
    text = bytearray.fromhex(hex_string).decode()
    print(text)
    return

def get_most_likely_byte(byte_list):
    if (len(byte_list) == 0):
        return "00"
    elif (len(byte_list) == 1):
        return byte_list[0]
    else:
        # If a byte appears two times in the list, choose that one.
        bytes_seen = []
        for byte in byte_list:
            if (byte in bytes_seen):
                return byte
            else:
                bytes_seen.append(byte)
        return "00"

def get_secret_key(secret_key_dict, length):
    secret_key = ""
    for i in range(length):
        secret_key += get_most_likely_byte(secret_key_dict[i])
    return secret_key

def print_decrypted_ciphertexts(ciphertexts, secret_key, preprocessing_function):
    for cipher in ciphertexts:
        hex_message = bytes.hex(bxor(bytes.fromhex(cipher), bytes.fromhex(secret_key)))
        print_text(hex_message, preprocessing_function)
    return

In [ ]:
ciphertexts = ["315c4eeaa8b5f8aaf9174145bf43e1784b8fa00dc71d885a804e5ee9fa40b16349c146fb778cdf2d3aff021dfff5b403b510d0d0455468aeb98622b137dae857553ccd8883a7bc37520e06e515d22c954eba5025b8cc57ee59418ce7dc6bc41556bdb36bbca3e8774301fbcaa3b83b220809560987815f65286764703de0f3d524400a19b159610b11ef3e",
               "234c02ecbbfbafa3ed18510abd11fa724fcda2018a1a8342cf064bbde548b12b07df44ba7191d9606ef4081ffde5ad46a5069d9f7f543bedb9c861bf29c7e205132eda9382b0bc2c5c4b45f919cf3a9f1cb74151f6d551f4480c82b2cb24cc5b028aa76eb7b4ab24171ab3cdadb8356f",
               "32510ba9a7b2bba9b8005d43a304b5714cc0bb0c8a34884dd91304b8ad40b62b07df44ba6e9d8a2368e51d04e0e7b207b70b9b8261112bacb6c866a232dfe257527dc29398f5f3251a0d47e503c66e935de81230b59b7afb5f41afa8d661cb",
               "32510ba9aab2a8a4fd06414fb517b5605cc0aa0dc91a8908c2064ba8ad5ea06a029056f47a8ad3306ef5021eafe1ac01a81197847a5c68a1b78769a37bc8f4575432c198ccb4ef63590256e305cd3a9544ee4160ead45aef520489e7da7d835402bca670bda8eb775200b8dabbba246b130f040d8ec6447e2c767f3d30ed81ea2e4c1404e1315a1010e7229be6636aaa",
               "3f561ba9adb4b6ebec54424ba317b564418fac0dd35f8c08d31a1fe9e24fe56808c213f17c81d9607cee021dafe1e001b21ade877a5e68bea88d61b93ac5ee0d562e8e9582f5ef375f0a4ae20ed86e935de81230b59b73fb4302cd95d770c65b40aaa065f2a5e33a5a0bb5dcaba43722130f042f8ec85b7c2070",
               "32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd2061bbde24eb76a19d84aba34d8de287be84d07e7e9a30ee714979c7e1123a8bd9822a33ecaf512472e8e8f8db3f9635c1949e640c621854eba0d79eccf52ff111284b4cc61d11902aebc66f2b2e436434eacc0aba938220b084800c2ca4e693522643573b2c4ce35050b0cf774201f0fe52ac9f26d71b6cf61a711cc229f77ace7aa88a2f19983122b11be87a59c355d25f8e4",
               "32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd90f1fa6ea5ba47b01c909ba7696cf606ef40c04afe1ac0aa8148dd066592ded9f8774b529c7ea125d298e8883f5e9305f4b44f915cb2bd05af51373fd9b4af511039fa2d96f83414aaaf261bda2e97b170fb5cce2a53e675c154c0d9681596934777e2275b381ce2e40582afe67650b13e72287ff2270abcf73bb028932836fbdecfecee0a3b894473c1bbeb6b4913a536ce4f9b13f1efff71ea313c8661dd9a4ce",
               "315c4eeaa8b5f8bffd11155ea506b56041c6a00c8a08854dd21a4bbde54ce56801d943ba708b8a3574f40c00fff9e00fa1439fd0654327a3bfc860b92f89ee04132ecb9298f5fd2d5e4b45e40ecc3b9d59e9417df7c95bba410e9aa2ca24c5474da2f276baa3ac325918b2daada43d6712150441c2e04f6565517f317da9d3",
               "271946f9bbb2aeadec111841a81abc300ecaa01bd8069d5cc91005e9fe4aad6e04d513e96d99de2569bc5e50eeeca709b50a8a987f4264edb6896fb537d0a716132ddc938fb0f836480e06ed0fcd6e9759f40462f9cf57f4564186a2c1778f1543efa270bda5e933421cbe88a4a52222190f471e9bd15f652b653b7071aec59a2705081ffe72651d08f822c9ed6d76e48b63ab15d0208573a7eef027",
               "466d06ece998b7a2fb1d464fed2ced7641ddaa3cc31c9941cf110abbf409ed39598005b3399ccfafb61d0315fca0a314be138a9f32503bedac8067f03adbf3575c3b8edc9ba7f537530541ab0f9f3cd04ff50d66f1d559ba520e89a2cb2a83",
               "32510ba9babebbbefd001547a810e67149caee11d945cd7fc81a05e9f85aac650e9052ba6a8cd8257bf14d13e6f0a803b54fde9e77472dbff89d71b57bddef121336cb85ccb8f3315f4b52e301d16e9f52f904"
              ]

# Get smallest ciphertext length
min_cipher_length = float("inf")
for cipher in ciphertexts:
    if (len(cipher) < min_cipher_length):
        min_cipher_length = len(cipher)

# Trim ciphers to smallest length
for i in range(len(ciphertexts)):
    ciphertexts[i] = ciphertexts[i][0:min_cipher_length]

# Get the number of characters in each cipher
cipher_char_count = int(min_cipher_length/2)

# Initialize secret key dictionary
secret_key_dict = {}
for i in range(cipher_char_count):
    secret_key_dict[i] = []

for i in range(len(ciphertexts)):
    chosen_cipher = ciphertexts[i]
    other_ciphertexts = ciphertexts.copy()
    other_ciphertexts.pop(i)
    
    xored_ciphertexts = []
    for cipher in other_ciphertexts:
        xor = bytes.hex(bxor(bytes.fromhex(cipher), bytes.fromhex(chosen_cipher)))
        xored_ciphertexts.append(xor)
    
    for j in range(cipher_char_count):
        az_char_count = 0
        for xor in xored_ciphertexts:
            # If the xors of the chosen cipher with several other ciphers have a character in [a-zA-Z]
            # at position j, the chosen cipher likely has a space in that position.
            if (is_az_char(xor[2*j:2*(j + 1)])):
                az_char_count += 1
        
        # TODO: remove magic number 4
        if (az_char_count >= len(xored_ciphertexts) - 4):
            # Calculate letter in ciphertexts[0] (this calculation assumes the chosen cipher has a space at position j).
            letter_at_j = bxor(bytes.fromhex(xored_ciphertexts[0][2*j:2*(j + 1)]), bytes.fromhex("20"))
            # Obtain one byte of the secret key using the letter.
            key_at_j = bytes.hex(bxor(bytes.fromhex(ciphertexts[0][2*j:2*(j + 1)]), letter_at_j))
            secret_key_dict[j].append(key_at_j)

print("---------- secret key ----------")
secret_key = get_secret_key(secret_key_dict, cipher_char_count)
print(secret_key)

# Decrypt ciphertexts. TODO: insert spaces where key is not known
print("---------- decrypted ciphertexts ----------")
print_decrypted_ciphertexts(ciphertexts, secret_key, replace_non_ascii_chars_with_space)



In [ ]:
# Calculate another byte of the secret key (this assumes we know a character of the original message).
i = 70
character = " "
byte_of_secret_key = bytes.hex(bxor(bytes.fromhex(ciphertexts[10][i:i + 2]), bytes([ord(character)])))
print(byte_of_secret_key)
secret_key = secret_key[0:i] + byte_of_secret_key + secret_key[i + 2:]

In [ ]:
print("---------- secret key ----------")
print(secret_key)

print("---------- decrypted ciphertexts ----------")
print_decrypted_ciphertexts(ciphertexts, secret_key, replace_non_ascii_chars_with_space)

In [ ]:
secret_key = "66396e89c9dbd8cc9874352acd6395102eafce78aa7fed28a07f6bc98d29c50b69b0339a19f8aa401a9c6d708f80c066c763fef0123148cdd8e802d05ba98777335daefcecd59c433a6b268b60bf4ef03c9a61"

print("---------- decrypted ciphertexts ----------")
print_decrypted_ciphertexts(ciphertexts, secret_key, identity)
